# Element - Base Class for lionpride Entities

Element is the foundation for all lionpride entities (Node, Event, Flow, etc.). It provides:

**Core Features:**
- **Immutable Identity**: Auto-generated UUID and UTC timestamp
- **Polymorphic Serialization**: Type-safe serialization/deserialization via `lion_class` metadata
- **Multi-Mode Serialization**: `python`/`json`/`db` modes for different contexts
- **Identity-Based Equality**: Elements equal by ID, not field values
- **Flexible Validation**: Accepts various UUID versions and timezones

In [1]:
import datetime as dt
from uuid import UUID, uuid4

from lionpride.core import Element

## 1. Basic Construction

Element auto-generates ID and timestamp with no required fields.

In [2]:
# Auto-generation
elem = Element()
print(f"ID: {elem.id}")
print(f"Created: {elem.created_at}")
print(f"Metadata: {elem.metadata}")

ID: c12ff4e2-04ba-467d-a0d5-c3e8c0abbba5
Created: 2025-11-23 19:04:45.752372+00:00
Metadata: {}


In [3]:
# Custom values
custom_elem = Element(
    id=uuid4(),
    created_at=dt.datetime(2025, 1, 1, 12, 0, 0, tzinfo=dt.UTC),
    metadata={"tag": "important", "version": 1},
)
print(custom_elem)

id=UUID('9e5e5a15-22fa-467d-8d9f-545e17a5a15d') created_at=datetime.datetime(2025, 1, 1, 12, 0, tzinfo=datetime.timezone.utc) metadata={'tag': 'important', 'version': 1}


## 2. Serialization Modes

Three modes for different contexts:
- `python`: Native types (UUID, datetime objects)
- `json`: JSON-safe strings
- `db`: Database mode (renames metadata → node_metadata)

In [4]:
# Python mode - native types
python_data = elem.to_dict(mode="python")
print(f"ID type: {type(python_data['id'])}")
print(f"Timestamp type: {type(python_data['created_at'])}")
print(f"lion_class injected: {'lion_class' in python_data['metadata']}")

ID type: <class 'uuid.UUID'>
Timestamp type: <class 'datetime.datetime'>
lion_class injected: True


In [5]:
# JSON mode - strings for API/storage
json_data = elem.to_dict(mode="json")
print(f"ID: {json_data['id']} (type: {type(json_data['id'])})")
print(f"Timestamp: {json_data['created_at']} (type: {type(json_data['created_at'])})")

# Verify strings parse back correctly
UUID(json_data["id"])
dt.datetime.fromisoformat(json_data["created_at"])
print("✓ Strings parse correctly")

ID: c12ff4e2-04ba-467d-a0d5-c3e8c0abbba5 (type: <class 'str'>)
Timestamp: 2025-11-23T19:04:45.752372+00:00 (type: <class 'str'>)
✓ Strings parse correctly


## 3. Polymorphic Deserialization

The `lion_class` metadata enables type-safe deserialization to the correct subclass.

In [6]:
# Define a subclass
class PersonElement(Element):
    name: str
    age: int


# Create and serialize
person = PersonElement(name="Alice", age=30)
data = person.to_dict(mode="python")
print(f"Serialized lion_class: {data['metadata']['lion_class']}")

Serialized lion_class: __main__.PersonElement


In [7]:
# Deserialize via base class - gets correct subclass
restored = Element.from_dict(data)
print(f"Type: {type(restored).__name__}")
print(f"Name: {restored.name}")
print(f"Age: {restored.age}")

# lion_class removed from metadata after deserialization
print(f"lion_class in metadata: {'lion_class' in restored.metadata}")

Type: PersonElement
Name: Alice
Age: 30
lion_class in metadata: False


## 4. Immutable Identity

ID and timestamp are frozen - cannot be changed after creation.

In [8]:
elem = Element()
original_id = elem.id

try:
    elem.id = uuid4()  # Attempt to change ID
except Exception as e:
    print(f"✓ Cannot change ID: {type(e).__name__}")

try:
    elem.created_at = dt.datetime.now(dt.UTC)  # Attempt to change timestamp
except Exception as e:
    print(f"✓ Cannot change timestamp: {type(e).__name__}")

# Metadata IS mutable
elem.metadata["new_key"] = "new_value"
print(f"✓ Metadata is mutable: {elem.metadata}")

✓ Cannot change ID: ValidationError
✓ Cannot change timestamp: ValidationError
✓ Metadata is mutable: {'new_key': 'new_value'}


## 5. Identity-Based Equality

Elements are equal if they have the same ID, regardless of other field values.

In [9]:
elem1 = Element(metadata={"value": 1})
elem2 = Element(id=elem1.id, metadata={"value": 999})  # Same ID, different metadata
elem3 = Element(metadata={"value": 1})  # Different ID, same metadata

print(f"elem1 == elem2 (same ID): {elem1 == elem2}")
print(f"elem1 == elem3 (different ID): {elem1 == elem3}")

# Hash also by ID - safe for sets/dicts
print(f"hash(elem1) == hash(elem2): {hash(elem1) == hash(elem2)}")

# Works in sets
element_set = {elem1, elem2, elem3}
print(f"Set size (elem1 and elem2 deduplicated): {len(element_set)}")

elem1 == elem2 (same ID): True
elem1 == elem3 (different ID): False
hash(elem1) == hash(elem2): True
Set size (elem1 and elem2 deduplicated): 2


## 6. Flexible Timestamp Handling

Element accepts naive datetimes (coerced to UTC) and any timezone.

In [10]:
import zoneinfo

# Naive datetime → coerced to UTC
naive_dt = dt.datetime(2025, 1, 1, 12, 0, 0)
elem_naive = Element(created_at=naive_dt)
print(f"Naive datetime coerced to UTC: {elem_naive.created_at.tzinfo}")

# Any timezone accepted
eastern = dt.datetime(2025, 1, 1, 12, 0, 0, tzinfo=zoneinfo.ZoneInfo("US/Eastern"))
elem_eastern = Element(created_at=eastern)
print(f"Eastern timezone preserved: {elem_eastern.created_at.tzinfo}")

Naive datetime coerced to UTC: UTC
Eastern timezone preserved: US/Eastern


## 7. Serialization Format Options

Control timestamp format and metadata key renaming.

In [11]:
elem = Element()

# Python mode with different timestamp formats
data_datetime = elem.to_dict(mode="python", created_at_format="datetime")
data_iso = elem.to_dict(mode="python", created_at_format="isoformat")
data_ts = elem.to_dict(mode="python", created_at_format="timestamp")

print(f"datetime: {type(data_datetime['created_at']).__name__}")
print(f"isoformat: {type(data_iso['created_at']).__name__} = {data_iso['created_at']}")
print(f"timestamp: {type(data_ts['created_at']).__name__} = {data_ts['created_at']:.2f}")

datetime: datetime
isoformat: str = 2025-11-23T19:04:45.792018+00:00
timestamp: float = 1763924685.79


In [12]:
# DB mode - renames metadata to node_metadata
db_data = elem.to_dict(mode="db")
print(f"Keys: {list(db_data.keys())}")
print(f"'metadata' in db_data: {'metadata' in db_data}")
print(f"'node_metadata' in db_data: {'node_metadata' in db_data}")

# Deserialize from db mode
restored = Element.from_dict(db_data, meta_key="node_metadata")
print(f"✓ Roundtrip successful: {restored.id == elem.id}")

Keys: ['id', 'created_at', 'node_metadata']
'metadata' in db_data: False
'node_metadata' in db_data: True
✓ Roundtrip successful: True


## 8. Roundtrip Fidelity

Serialization and deserialization preserve identity across all modes.

In [13]:
original = Element(metadata={"test": "value", "count": 42})

# Python mode roundtrip
python_restored = Element.from_dict(original.to_dict(mode="python"))
print(f"Python roundtrip: ID matches = {python_restored.id == original.id}")
print(f"Python roundtrip: Metadata matches = {python_restored.metadata == original.metadata}")

# JSON mode roundtrip
json_restored = Element.from_dict(original.to_dict(mode="json"))
print(f"JSON roundtrip: ID matches = {json_restored.id == original.id}")
print(f"JSON roundtrip: Metadata matches = {json_restored.metadata == original.metadata}")

# from_json convenience method
json_str = original.to_json()
json_method_restored = Element.from_json(json_str)
print(f"from_json roundtrip: ID matches = {json_method_restored.id == original.id}")

Python roundtrip: ID matches = True
Python roundtrip: Metadata matches = True
JSON roundtrip: ID matches = True
JSON roundtrip: Metadata matches = True
from_json roundtrip: ID matches = True


## Summary Checklist

**Element Essentials:**
- ✅ Auto-generates UUID and UTC timestamp
- ✅ Three serialization modes: `python`, `json`, `db`
- ✅ Polymorphic deserialization via `lion_class` metadata
- ✅ Immutable identity (frozen ID and timestamp)
- ✅ Identity-based equality and hashing
- ✅ Flexible timestamp handling (naive → UTC, any timezone)
- ✅ Mutable metadata for workflow state
- ✅ Lossless roundtrip through serialization

**Next Steps:**
- See `Node` for content-bearing elements
- See `Event` for state change tracking
- See `Flow` for workflow orchestration